A tener en cuenta:

TS: falta automatizar deteccion de cantidad de planillas en el archivo. falta documentar
dexa: Ok proceso de descarga. falta documentar
Selnet: EL proceso esta OK, tengo problemas para cambiarle el nombre al archivo (se rompe si lo hago)

In [4]:
import selenium
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import os
from datetime import datetime
import zipfile
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import json
from openpyxl import Workbook
import re
import pickle

In [23]:
# Obtén el directorio de trabajo actual
project_folder = os.getcwd()
download_folder = os.path.join(project_folder, "C:\\Users\\Juan M Cabral\\Desktop\\Pycharm\\lista_de_precios_axon\\rawdata")

In [3]:
# Ruta al archivo JSON de las credenciales de tu cuenta de servicio
credentials_file = 'listadeprecios-rocnet-2ebb9564fd3b.json'

In [4]:
#TS y SAWERIN

In [26]:

# Lista de enlaces a los spreadsheets de Google Sheets
links = [
    'https://docs.google.com/spreadsheets/d/1E9oFbOPCYIEqZO7rOOC9Qbk46HwgGO-V_qwLaww5USY/edit#gid=1458627182',
    'https://docs.google.com/spreadsheets/d/1UQpngaO_VisWIe8xel0Kp5uuq21COWFF-W7deek6oXs/edit#gid=2'
]

# Nombres de archivo correspondientes a cada enlace
file_names = ['ts.xlsx', 'sawerin.xlsx']

# Autenticación del cliente utilizando las credenciales de tu cuenta de servicio
credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_file)
client = gspread.authorize(credentials)

# Función para reemplazar caracteres no válidos en el título de la hoja de cálculo
def clean_sheet_title(title):
    cleaned_title = re.sub(r'[\\/?:\[\]]', '', title)
    return cleaned_title[:31]

# Procesar cada enlace y generar los archivos correspondientes
for i, link in enumerate(links):
    # Abrir el spreadsheet y obtener la lista de hojas de cálculo
    spreadsheet = client.open_by_url(link)
    sheet_list = spreadsheet.worksheets()

    # Crear un nuevo libro de Excel
    workbook = Workbook()

    # Para cada hoja de cálculo en sheet_list
    for sheet in sheet_list:
        # Obtener un título limpio para la hoja de cálculo
        sheet_title = clean_sheet_title(sheet.title)

        # Crear una nueva hoja en el libro de Excel con el título limpio
        worksheet = workbook.create_sheet(title=sheet_title)

        # Obtener los valores de la hoja de cálculo
        values = sheet.get_all_values()

        # Llenar la hoja de cálculo con los valores
        for row in values:
            worksheet.append(row)

    # Eliminar la hoja de cálculo por defecto (Sheet)
    default_sheet = workbook['Sheet']
    workbook.remove(default_sheet)

    # Obtener el nombre del archivo basado en el enlace
    filename = file_names[i]
    
    # Ruta completa del archivo de destino
    file_path = os.path.join(download_folder, filename)

    # Guardar el libro de Excel en un archivo .xlsx
    workbook.save(file_path)



In [6]:
with open("credenciales.json") as archivo:
    credenciales = json.load(archivo)

In [17]:
user = credenciales["user"]
password = credenciales["password"]

# Configurar el Webdriver
options = Options()
options.add_experimental_option("detach", True)
prefs = {
    "download.default_directory": download_folder
}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

driver.get("https://www.dexa.net.ar/productos/catalogo")
driver.implicitly_wait(3)

user_input = driver.find_element(by=By.NAME, value="data[Usuario][login]")
user_input.send_keys(user)

user_input = driver.find_element(by=By.NAME, value="data[Usuario][password]")
user_input.send_keys(password)

driver.find_element(by=By.XPATH, value='//button[@class="btn btn-primary" and text()="Ingresar"]').click()
time.sleep(4)

dropdown = driver.find_element(by=By.LINK_TEXT, value='Productos')

# Hacer clic en el elemento de lista desplegable
dropdown.click()

# Esperar a que aparezcan las opciones del menú desplegable
time.sleep(3)

driver.find_element(by=By.XPATH, value='//a[contains(text(), "Exportar lista de precios")]').click()
time.sleep(3)
driver.find_element(By.XPATH, '//button[contains(.,"Descargar lista de precios")]').click()
time.sleep(15)

fecha_actual = datetime.now().strftime("%d_%m_%Y")

# Especificar el nombre del archivo descargado
nombre_archivo_descargado = "Lista Precios DEXA Seguridad Electrónica S.R.L. {}.xlsx".format(fecha_actual)

# Esperar a que el archivo se descargue completamente
while not os.path.exists(os.path.join(download_folder, nombre_archivo_descargado)):
    time.sleep(1)

# Obtener la ruta completa del archivo descargado
downloaded_file_path = os.path.join(download_folder, nombre_archivo_descargado)

# Renombrar el archivo descargado a "dexa.xlsx"
nuevo_nombre_archivo = "dexa.xlsx"
nueva_ruta_archivo = os.path.join(download_folder, nuevo_nombre_archivo)
os.rename(downloaded_file_path, nueva_ruta_archivo)



In [ ]:
#SELNET

In [ ]:
#CONFIGURAR EL NOMBRE PARA LA DESCARGA DEL ARCHIVO!

In [29]:
# URL del archivo zip en Dropbox
url = "https://www.dropbox.com/sh/c1y13dqqnqwgpfk/AADmt_8B5LUPatOkehCuIfrTa?dl=1"

# Realizar la solicitud GET para obtener el contenido del archivo zip
response = requests.get(url)

# Guardar el contenido del archivo zip en disco
file_path = os.path.join(download_folder, "archivo.zip")
with open(file_path, "wb") as file:
    file.write(response.content)

# Descomprimir el archivo zip en la carpeta de destino
with zipfile.ZipFile(file_path, "r") as zip_ref:
    zip_ref.extractall(download_folder)



'C:\\Users\\Juan M Cabral\\Desktop\\Pycharm\\lista_de_precios_axon\\rawdata\\selnet.xlsx'